

1.   Nukula anjani rahul(Roll number:2010110966)
2.   k sai srinivas(Roll number:2010110833)


In [88]:
import nltk
import string
import re
import glob
import math
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# read corpus

In [89]:
from google.colab import drive
drive.mount('/content/drive')
def read_file(file_name): 
  with open('/content/drive/MyDrive/corpus2/{}'.format(file_name)) as f:
      lines = f.read()
  return lines
file_list = glob.glob("/content/drive/MyDrive/corpus2/*")
for i in range(len(file_list)):
  name = file_list[i].split("/")
  file_list[i] = name[-1]
doc_id = {}
for i in range(len(file_list)):
  doc_id[i] = file_list[i]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Processing

In [90]:
nltk_stopwords = nltk.corpus.stopwords.words('english')

def preprocess(text):
  text = text.lower()
  text = "".join([char for char in text if char not in string.punctuation])
  text = re.sub('[0-9]+', '', text)
  text = text.strip()
  tokens = re.split('\W+', text)
  tokens = [token for token in tokens if token != '' and len(token) > 1]
  tokens = [token for token in tokens if token not in nltk_stopwords]
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(token) for token in tokens]
  stemmer = PorterStemmer()
  tokens = [stemmer.stem(token) for token in tokens]
  return tokens

## Doc Freq Term Index Term frequency

In [91]:
tf_doc,tf_weight,normalized,inverted_index,df_index,tf_index = {},{},{},{},{},{}
for i in range(len(file_list)):
  per_doc = {}
  lines = read_file(file_list[i])
  tokens=preprocess(lines)
  for word in tokens:
    if word in inverted_index and i not in inverted_index[word]:
      inverted_index[word].append(i)
    elif word not in inverted_index.keys():
      inverted_index[word] = [i]  
    if word in tf_index :
      tf_index[word] = tf_index[word] +1
    elif word not in tf_index.keys():
      tf_index[word] = 1  
    if word in per_doc:
      per_doc[word] = per_doc[word] + 1
    else:
      per_doc[word] = 1  
  tf_doc[i] = per_doc        
  tokens = list(set(tokens))
  for word in tokens:
    if word in df_index.keys() :
      df_index[word] = df_index[word] + 1
    elif word not in df_index.keys():
      df_index[word] = 1
for i in range(len(tf_doc)):
  tf_weight_doc = tf_doc[i]
  for key in tf_weight_doc.keys():
    tf_weight_doc[key] = 1 + math.log10(tf_weight_doc[key])
  tf_weight[i] = tf_weight_doc
for i in range(len(tf_weight)):
  normailized_doc = tf_weight[i]
  squared_sum=0
  for value in normailized_doc.values():
    squared_sum = squared_sum + (value**2)
  normalization_factor = math.sqrt(squared_sum)
  for key in normailized_doc.keys():
    normailized_doc[key] = normailized_doc[key]/normalization_factor
  normalized[i] = normailized_doc
print(normalized )

{0: {'messeng': 0.1142551777202966, 'user': 0.09875807046122695, 'simpli': 0.07145986463749557, 'download': 0.06294095835559164, 'app': 0.1151493640741344, 'mobiletablet': 0.04837779187671203, 'devic': 0.04837779187671203, 'also': 0.08602303111637519, 'use': 0.09454193739827911, 'desktop': 0.04837779187671203, 'option': 0.07145986463749557, 'set': 0.09675558375342407, 'facebook': 0.11131875023230368, 'account': 0.08219241727454446, 'telephon': 0.04837779187671203, 'number': 0.06294095835559164, 'similar': 0.06294095835559164, 'whatsapp': 0.06294095835559164, 'viber': 0.04837779187671203, 'oovoo': 0.04837779187671203, 'etc': 0.06294095835559164, 'choos': 0.06294095835559164, 'setup': 0.04837779187671203, 'profil': 0.08219241727454446, 'take': 0.06294095835559164, 'inform': 0.07750412483447126, 'includ': 0.07145986463749557, 'friend': 0.08926176897193902, 'list': 0.07145986463749557, 'pictur': 0.04837779187671203, 'sync': 0.07750412483447126, 'phone': 0.07145986463749557, 'contact': 0.09

# Query Processing

In [95]:
query = "The Binance Exchange is a leading cryptocurrency"
query = re.sub(r"\s+"," ",query) 
query = query.split(' ')  
query_tokens,df_query_terms,query_doc_list,query_tf,query_tf_weight,query_final_weights =[],{},{},{},{},{}
for i in range(len(query)):
  lines = query[i]
  token=preprocess(lines)
  if token != []:
    query_tokens.append(token[0])
# print("Query tokens")
# print(query_tokens)
for word in query_tokens:
  if word not in df_index:
    query_tokens.remove(word)
for token in query_tokens:
  value = df_index[token]
  df_query_terms[token] = value
# print("DF query terms")
# print(df_query_terms)
for key in query_tokens:
  query_doc_list[key] = inverted_index[key]
# print("Query document list")
# print(query_doc_list)
for word in query_tokens:
  if word in query_tf:
    query_tf[word] = query_tf[word] + 1
  else:
    query_tf[word] = 1
# print("Query term frequeny")
# print(query_tf)
for word in query_tf:
  query_tf_weight[word] = 1 + math.log10(query_tf[word])
# print("Query Term weight frequency")
# print(query_tf_weight)
idf_query_terms = {} 
for word in df_query_terms:
  idf_query_terms[word] = math.log10(len(doc_id)/df_query_terms[word])
# print("IDF")
# print(idf_query_terms)
for word in query_tokens:
  query_final_weights[word] = query_tf[word] * idf_query_terms[word]
print("Query final weights")
print(query_final_weights)

Query final weights
{'binanc': 1.6127838567197355, 'exchang': 0.8346326063360918, 'lead': 0.7096938697277919, 'cryptocurr': 1.6127838567197355}


In [96]:
result_doc_list = {} 
for word in query_doc_list:
  for doc_num in query_doc_list[word]:
    doc_weight = tf_weight[doc_num][word] * query_final_weights[word]
    if doc_num in result_doc_list:
      result_doc_list[doc_num] = result_doc_list[doc_num] + doc_weight
    else:
      result_doc_list[doc_num] = doc_weight
print(result_doc_list)

{32: 0.533484416846283, 1: 0.04778934190358737, 15: 0.060514744754028654, 21: 0.04320845130005389, 38: 0.08964706008366183, 39: 0.02933379682897512, 8: 0.033915234964738514, 16: 0.030920152834978083, 17: 0.04303797865512978, 30: 0.04014173019438955, 31: 0.04305526625793404, 37: 0.03718321369350137}


In [97]:
final_query_docs = sorted(result_doc_list, key=result_doc_list.get, reverse=True)
output=[]
if len(final_query_docs)>10:
  final_query_docs = final_query_docs[:10]
for i in final_query_docs:
  file_num = i
  output.append(doc_id[file_num])
print(output)  

['Binance.txt', 'puma.txt', 'nokia.txt', 'paypal.txt', 'Lenovo.txt', 'motorola.txt', 'microsoft.txt', 'flipkart.txt', 'Uber.txt', 'whatsapp.txt']
